# 1. 데이터 불러오기
- y값: g191a297 (A52. <취업자 공통> <이직 준비 관련> 이직준비를 하고 있는지 여부)

In [ ]:
conda install -c conda-forge imbalanced-learn

In [ ]:
pip install bayesian-optimization

In [ ]:
pip install bayesian-optimization

In [ ]:
pip install SHAP

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import re
import semopy
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from semopy import Model, report
import graphviz
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pydot
from IPython.display import Image, display
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import ADASYN, BorderlineSMOTE, SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from bayes_opt import BayesianOptimization

# 7. 모델링 - 예측 모델

## 7-1. SMOTE + 구조방정식 변수 20-> 23개

In [ ]:
import os

#현재 폴더 경로; 작업 폴더 기준
print(os.getcwd())

In [ ]:
base_path = os.getcwd()

In [ ]:
# clean_done_df 불러오기
clean_done_df = pd.read_csv(f'{base_path}/data/clean_done_df.csv', encoding='EUC-KR', low_memory=False)

In [ ]:
clean_done_df = clean_done_df.drop(columns= ['Y191ear'])

In [ ]:
# estimates
estimates = pd.read_csv(f'{base_path}/data/sem_model_estimates.csv',encoding='EUC-KR',low_memory=False)

In [ ]:
# params 데이터프레임이 이미 있는 상태에서 진행합니다.
params = estimates

# 중요 변수 상위 20개 필터링 (요인 적재값 기준)
important_params = params.loc[params['op'].isin(['~', '=~'])].copy()
important_params['abs_estimate'] = important_params['Estimate'].abs()
top_20_params = important_params.nlargest(23, 'abs_estimate') # 변수 개수 선택

# 상위 20개의 변수명 추출
top_20_vars = top_20_params['lval'].unique()

In [ ]:
top_20_vars

In [ ]:
y_counts = clean_done_df['g191a297'].value_counts()
print(y_counts)

### SMOTE 불균형 처리

In [ ]:
# clean_done_df 불러오기
clean_done_df = pd.read_csv(f'{base_path}/data/clean_done_df.csv', encoding='EUC-KR', low_memory=False)

In [ ]:
X_top_20 = clean_done_df[top_20_vars]
y = clean_done_df['g191a297']  # 타겟 변수 설정

In [ ]:
X_top_20

In [ ]:
# SMOTE 적용
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_top_20, y)

# resampled 데이터프레임 생성
resampled_df = pd.DataFrame(X_resampled, columns=X_resampled.columns)
resampled_df['g191a297'] = y_resampled

# 클래스 0과 1의 개수 출력
class_counts = resampled_df['g191a297'].value_counts()
print(class_counts)

In [ ]:
X_resampled = X_resampled.drop(columns= ['g191a297'])

In [ ]:
X_resampled.info()

In [ ]:
X_resampled

In [ ]:
# 학습 및 테스트 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# 모델 리스트
models = {
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'XGBoost': xgb.XGBClassifier(),
    'LightGBM': lgb.LGBMClassifier()
}

# 모델 학습 및 평가 결과 저장을 위한 파일
results = []

# 모델 학습 및 평가
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results.append(f'{model_name} Accuracy: {accuracy:.4f}\n{report}\n')

# 결과를 txt 파일로 저장
with open('result/Tree-based_model_results_SMOTE_23.txt', 'w') as f:
    for result in results:
        f.write(result)
        f.write('\n')

print("Tree-based (SMOTE): Results have been saved to 'Tree-based_model_results_SMOTE_23.txt'.")

In [ ]:
# Ensure the data is in the correct format and contiguous in memory
X_top_20 = np.ascontiguousarray(X_top_20, dtype=np.float64)
y = np.ascontiguousarray(y, dtype=np.float64)

# 학습 및 테스트 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# 모델 리스트
models = {
    'KNN': KNeighborsClassifier(),
}

# 모델 학습 및 평가 결과 저장을 위한 파일
results = []

# 모델 학습 및 평가
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results.append(f'{model_name} Accuracy: {accuracy:.4f}\n{report}\n')

# 결과를 txt 파일로 저장
with open('result/SVM.KNNL_model_results_SMOTE_23.txt', 'w') as f:
    for result in results:
        f.write(result)
        f.write('\n')

print("SVM.KNN (SMOTE): Results have been saved to 'SVM.KNNL_model_results_SMOTE_23.txt'.")

In [ ]:
# 학습 및 테스트 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# 모델 리스트
models = {
    'Logistic Regression': LogisticRegression(),
    'Naive Bayes': GaussianNB()
}

# 모델 학습 및 평가 결과 저장을 위한 파일
results = []

# 모델 학습 및 평가
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results.append(f'{model_name} Accuracy: {accuracy:.4f}\n{report}\n')

# 결과를 txt 파일로 저장
with open('result/LR.NB_model_results_SMOTE_23.txt', 'w') as f:
    for result in results:
        f.write(result)
        f.write('\n')

print("LR.NB (SMOTE): Results have been saved to 'LR.NB_model_results_SMOTE_23.txt'.")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# 베이지안 옵티마이제이션
def optimize_model(model, param_bounds):
    def model_evaluate(**params):
        for param in params:
            if param in ['max_depth', 'min_samples_split', 'n_estimators', 'num_leaves', 'n_neighbors', 'leaf_size']:
                params[param] = int(params[param])
        model.set_params(**params)
        return cross_val_score(model, X_train, y_train, scoring='accuracy', cv=5).mean()

    optimizer = BayesianOptimization(f=model_evaluate, pbounds=param_bounds, random_state=42)
    optimizer.maximize(init_points=10, n_iter=20)
    
    return optimizer.max['params']

# 모델 리스트와 파라미터 범위
models = {
    'Decision Tree': (DecisionTreeClassifier(), {
        'max_depth': (1, 1000),
        'min_samples_split': (2, 500) 
    }),
    'Random Forest': (RandomForestClassifier(), {
        'n_estimators': (10, 2000),
        'max_depth': (1, 1000),
        'min_samples_split': (2, 500)
    }),
    'XGBoost': (xgb.XGBClassifier(), {
        'n_estimators': (10, 2000),
        'max_depth': (1, 200), 
        'learning_rate': (0.001, 0.5)
    }),
    'LightGBM': (lgb.LGBMClassifier(), {
        'n_estimators': (10, 2000), 
        'num_leaves': (20, 1000),
        'learning_rate': (0.001, 0.5) 
    }),
    'KNN': (KNeighborsClassifier(), {
        'n_neighbors': (1, 500), 
        'leaf_size': (10, 1000) 
    }),
    'Logistic Regression': (LogisticRegression(), {
        'C': (0.001, 1000),
        
    }),
    'Naive Bayes': (GaussianNB(), {
        'var_smoothing': (1e-12, 1e-6)
    })
}



# 모델 학습 및 평가 결과 저장을 위한 파일
results = []

# 모델 학습 및 평가
for model_name, (model, param_bounds) in models.items():
    print(f"Tuning {model_name}...")
    best_params = optimize_model(model, param_bounds)
    # 정수형 파라미터를 다시 정수로 변환
    for param in best_params:
        if param in ['max_depth', 'min_samples_split', 'n_estimators', 'num_leaves', 'n_neighbors', 'leaf_size']:
            best_params[param] = int(best_params[param])
    model.set_params(**best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results.append(f'{model_name} Accuracy: {accuracy:.4f}\n{report}\nBest Params: {best_params}\n')

# 결과를 txt 파일로 저장
with open('result/model_results_SMOTE_23_optimized.txt', 'w') as f:
    for result in results:
        f.write(result)
        f.write('\n')

print("All models (SMOTE): Results have been saved to 'model_results_SMOTE_23_optimized.txt'.")


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# 베이지안 옵티마이제이션
def optimize_model(model, param_bounds):
    def model_evaluate(**params):
        for param in params:
            if param in ['max_depth', 'min_samples_split', 'n_estimators', 'num_leaves', 'n_neighbors', 'leaf_size']:
                params[param] = int(params[param])
        model.set_params(**params)
        return cross_val_score(model, X_train, y_train, scoring='accuracy', cv=5).mean()

    optimizer = BayesianOptimization(f=model_evaluate, pbounds=param_bounds, random_state=42)
    optimizer.maximize(init_points=10, n_iter=20)
    
    return optimizer.max['params']

# 모델 리스트와 파라미터 범위
models = {
    'Decision Tree': (DecisionTreeClassifier(), {
        'max_depth': (1, 1000),
        'min_samples_split': (2, 500) 
    }),
    'Random Forest': (RandomForestClassifier(), {
        'n_estimators': (10, 2000),
        'max_depth': (1, 1000),
        'min_samples_split': (2, 500)
    }),
    'XGBoost': (xgb.XGBClassifier(), {
        'n_estimators': (10, 2000),
        'max_depth': (1, 200), 
        'learning_rate': (0.001, 0.5)
    }),
    'LightGBM': (lgb.LGBMClassifier(), {
        'n_estimators': (10, 2000), 
        'num_leaves': (20, 1000),
        'learning_rate': (0.001, 0.5) 
    }),
    'KNN': (KNeighborsClassifier(), {
        'n_neighbors': (1, 500), 
        'leaf_size': (10, 1000) 
    }),
    'Logistic Regression': (LogisticRegression(), {
        'C': (0.001, 1000),
        
    }),
    'Naive Bayes': (GaussianNB(), {
        'var_smoothing': (1e-12, 1e-6)
    })
}



# 모델 학습 및 평가 결과 저장을 위한 파일
results = []

# 모델 학습 및 평가
for model_name, (model, param_bounds) in models.items():
    print(f"Tuning {model_name}...")
    best_params = optimize_model(model, param_bounds)
    # 정수형 파라미터를 다시 정수로 변환
    for param in best_params:
        if param in ['max_depth', 'min_samples_split', 'n_estimators', 'num_leaves', 'n_neighbors', 'leaf_size']:
            best_params[param] = int(best_params[param])
    model.set_params(**best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results.append(f'{model_name} Accuracy: {accuracy:.4f}\n{report}\nBest Params: {best_params}\n')

# 결과를 txt 파일로 저장
with open('result/model_results_SMOTE_23_optimized.txt', 'w') as f:
    for result in results:
        f.write(result)
        f.write('\n')

print("All models (SMOTE): Results have been saved to 'model_results_SMOTE_23_optimized.txt'.")


In [ ]:
import shap
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import numpy as np

# Function to plot and save SHAP values
def plot_shap(model, X_train, model_name):
    try:
        if isinstance(model, (DecisionTreeClassifier, RandomForestClassifier, xgb.XGBClassifier, lgb.LGBMClassifier)):
            explainer = shap.TreeExplainer(model)
        else:
            explainer = shap.Explainer(model, X_train)
        shap_values = explainer(X_train)
        shap.summary_plot(shap_values, X_train, show=False)
        plt.title(f"SHAP Summary Plot for {model_name}")
        plt.savefig(f"result/SHAP_{model_name}.png")
        plt.close()
    except Exception as e:
        print(f"Could not create SHAP plot for {model_name}: {str(e)}")

# 학습 및 테스트 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# 모델 리스트
tree_based_models = {
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'XGBoost': xgb.XGBClassifier(),
    'LightGBM': lgb.LGBMClassifier()
}

# 모델 학습 및 평가 결과 저장을 위한 파일
results = []

# 모델 학습 및 평가
for model_name, model in tree_based_models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results.append(f'{model_name} Accuracy: {accuracy:.4f}\n{report}\n')
    plot_shap(model, X_train, model_name)

# 결과를 txt 파일로 저장
with open('result/Tree-based_SHAP_results_SMOTE_23.txt', 'w') as f:
    for result in results:
        f.write(result)
        f.write('\n')

print("Tree-based (SMOTE): Results have been saved to 'Tree-based_model_results_SMOTE_23.txt'.")

# 학습 및 테스트 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# 모델 리스트
knn_model = {
    'KNN': KNeighborsClassifier(),
}

# 모델 학습 및 평가 결과 저장을 위한 파일
results = []

# 모델 학습 및 평가
for model_name, model in knn_model.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results.append(f'{model_name} Accuracy: {accuracy:.4f}\n{report}\n')
    plot_shap(model, X_train, model_name)

# 결과를 txt 파일로 저장
with open('result/SVM.KNNL_SHAP_results_SMOTE_23.txt', 'w') as f:
    for result in results:
        f.write(result)
        f.write('\n')

print("SVM.KNN (SMOTE): Results have been saved to 'SVM.KNNL_model_results_SMOTE_23.txt'.")

# 학습 및 테스트 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# 모델 리스트
linear_models = {
    'Logistic Regression': LogisticRegression(),
    'Naive Bayes': GaussianNB()
}

# 모델 학습 및 평가 결과 저장을 위한 파일
results = []

# 모델 학습 및 평가
for model_name, model in linear_models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results.append(f'{model_name} Accuracy: {accuracy:.4f}\n{report}\n')
    plot_shap(model, X_train, model_name)

# 결과를 txt 파일로 저장
with open('result/LR.NBS_SHAP_results_SMOTE_23.txt', 'w') as f:
    for result in results:
        f.write(result)
        f.write('\n')

print("LR.NB (SMOTE): Results have been saved to 'LR.NB_model_results_SMOTE_23.txt'.")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

def plot_learning_curve(model, X, y, model_name, train_sizes=np.linspace(0.1, 1.0, 5)):
    train_sizes, train_scores, test_scores = learning_curve(
        model, X, y, train_sizes=train_sizes, cv=None, n_jobs=-1, scoring='accuracy'
    )
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    plt.figure()
    plt.title(f"Learning Curve for {model_name}")
    plt.xlabel("Training steps")
    plt.ylabel("Score")
    plt.grid()
    
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Test score")
    
    plt.legend(loc="best")
    plt.savefig(f"result/Learning_Curve_{model_name}.png")
    plt.close()

# 학습 및 테스트 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# 모델 리스트
tree_based_models = {
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'XGBoost': xgb.XGBClassifier(),
    'LightGBM': lgb.LGBMClassifier()
}

# 모델 학습 및 평가 결과 저장을 위한 파일
results = []

# 모델 학습 및 평가
for model_name, model in tree_based_models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results.append(f'{model_name} Accuracy: {accuracy:.4f}\n{report}\n')
    plot_shap(model, X_train, model_name)
    plot_learning_curve(model, X_train, y_train, model_name)

# 결과를 txt 파일로 저장
with open('result/Tree-based_Curve_results_SMOTE_23.txt', 'w') as f:
    for result in results:
        f.write(result)
        f.write('\n')

print("Tree-based (SMOTE): Results have been saved to 'Tree-based_SHAP_results_SMOTE_23.txt'.")

# 학습 및 테스트 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# 모델 리스트
knn_model = {
    'KNN': KNeighborsClassifier(),
}

# 모델 학습 및 평가 결과 저장을 위한 파일
results = []

# 모델 학습 및 평가
for model_name, model in knn_model.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results.append(f'{model_name} Accuracy: {accuracy:.4f}\n{report}\n')
    plot_shap(model, X_train, model_name)
    plot_learning_curve(model, X_train, y_train, model_name)

# 결과를 txt 파일로 저장
with open('result/SVM.KNNL_Curve_results_SMOTE_23.txt', 'w') as f:
    for result in results:
        f.write(result)
        f.write('\n')

print("SVM.KNN (SMOTE): Results have been saved to 'SVM.KNNL_SHAP_results_SMOTE_23.txt'.")

# 학습 및 테스트 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# 모델 리스트
linear_models = {
    'Logistic Regression': LogisticRegression(),
    'Naive Bayes': GaussianNB()
}

# 모델 학습 및 평가 결과 저장을 위한 파일
results = []

# 모델 학습 및 평가
for model_name, model in linear_models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results.append(f'{model_name} Accuracy: {accuracy:.4f}\n{report}\n')
    plot_shap(model, X_train, model_name)
    plot_learning_curve(model, X_train, y_train, model_name)

# 결과를 txt 파일로 저장
with open('result/LR.NBS_Curve_results_SMOTE_23.txt', 'w') as f:
    for result in results:
        f.write(result)
        f.write('\n')

print("LR.NB (SMOTE): Results have been saved to 'LR.NB_SHAP_results_SMOTE_23.txt'.")


# 전체 변수+ SMOTE

In [ ]:
# clean_done_df 불러오기
clean_done_df = pd.read_csv(f'{base_path}/data/clean_done_df.csv', encoding='EUC-KR', low_memory=False)

In [ ]:
X = clean_done_df.drop(columns=['g191a297', 'Y191ear'])
y = clean_done_df['g191a297']  # 타겟 변수 설정

In [ ]:
X

In [ ]:
# SMOTE 적용
smote = SMOTE()
X, y = smote.fit_resample(X, y)

#데이터프레임 생성
df = pd.DataFrame(X, columns=X.columns)
df['g191a297'] = y

# 클래스 0과 1의 개수 출력
class_counts = df['g191a297'].value_counts()
print(class_counts)

In [ ]:
X = X.drop(columns= ['g191a297'])

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import xgboost as xgb
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 학습 및 테스트 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 모델 리스트
models = {
    'Logistic Regression': LogisticRegression(),
    'Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'XGBoost': xgb.XGBClassifier(),
    'LightGBM': lgb.LGBMClassifier()
}

# 모델 학습 및 평가 결과 저장을 위한 파일
results = []

# 모델 학습 및 평가
for model_name, model in models.items():
    # 교차 검증을 통한 모델 평가
    cross_val_scores = cross_val_score(model, X_train, y_train, cv=5)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    train_accuracy = accuracy_score(y_train, model.predict(X_train))
    test_accuracy = accuracy_score(y_test, y_pred)
    
    report = classification_report(y_test, y_pred)
    
    results.append(f'{model_name} Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}\n')
    results.append(f'Cross Validation Scores: {cross_val_scores}\n')
    results.append(f'Classification Report:\n{report}\n')

# 결과를 txt 파일로 저장
with open('result/Training.txt', 'w') as f:
    for result in results:
        f.write(result)
        f.write('\n')

print("saved")


In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

def plot_learning_curve(estimator, title, X, y, cv=None, n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    plt.xlabel("Training examples")
    plt.ylabel("")
    
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    plt.grid()
    
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")
    
    plt.legend(loc="best")
    return plt

for model_name, model in models.items():
    plot_learning_curve(model, f'Learning Curve for {model_name}', X, y, cv=5)

plt.show()


In [ ]:
# Ensure the data is in the correct format and contiguous in memory
X = np.ascontiguousarray(X, dtype=np.float64)
y = np.ascontiguousarray(y, dtype=np.float64)

# 학습 및 테스트 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 모델 리스트
models = {
    'SVM': SVC(probability=True),
    'KNN': KNeighborsClassifier(),
}

# 모델 학습 및 평가 결과 저장을 위한 파일
results = []

# 모델 학습 및 평가
for model_name, model in models.items():
    # 교차 검증을 통한 모델 평가
    cross_val_scores = cross_val_score(model, X_train, y_train, cv=5)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    train_accuracy = accuracy_score(y_train, model.predict(X_train))
    test_accuracy = accuracy_score(y_test, y_pred)
    
    report = classification_report(y_test, y_pred)
    
    results.append(f'{model_name} Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}\n')
    results.append(f'Cross Validation Scores: {cross_val_scores}\n')
    results.append(f'Classification Report:\n{report}\n')

# 결과를 txt 파일로 저장
with open('result/Training.txt', 'w') as f:
    for result in results:
        f.write(result)
        f.write('\n')

print("saved")


In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

def plot_learning_curve(estimator, title, X, y, cv=None, n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    plt.xlabel("Training examples")
    plt.ylabel("Accuracy")
    
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    plt.grid()
    
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")
    
    plt.legend(loc="best")
    return plt

for model_name, model in models.items():
    plot_learning_curve(model, f'Learning Curve for {model_name}', X, y, cv=5)

plt.show()

In [ ]:
# 학습 및 테스트 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 모델 리스트
models = {

    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'XGBoost': xgb.XGBClassifier(),
    'LightGBM': lgb.LGBMClassifier()
}

# 모델 학습 및 평가 결과 저장을 위한 파일
results = []

# 모델 학습 및 평가
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results.append(f'{model_name} Accuracy: {accuracy:.4f}\n{report}\n')

# 결과를 txt 파일로 저장
with open('result/Tree-based_model_results_SMOTE.txt', 'w') as f:
    for result in results:
        f.write(result)
        f.write('\n')

print("Tree-based (SMOTE): Results have been saved to 'Tree-based_model_results_SMOTE.txt'.")

In [ ]:
# Ensure the data is in the correct format and contiguous in memory
X = np.ascontiguousarray(X, dtype=np.float64)
y = np.ascontiguousarray(y, dtype=np.float64)

# 학습 및 테스트 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 모델 리스트
models = {
    'KNN': KNeighborsClassifier(),
}

# 모델 학습 및 평가 결과 저장을 위한 파일
results = []

# 모델 학습 및 평가
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results.append(f'{model_name} Accuracy: {accuracy:.4f}\n{report}\n')

# 결과를 txt 파일로 저장
with open('result/SVM.KNNL_model_results_SMOTE.txt', 'w') as f:
    for result in results:
        f.write(result)
        f.write('\n')

print("KNN (SMOTE): Results have been saved to 'SVM.KNNL_model_results_SMOTE.txt'.")

In [ ]:
# 학습 및 테스트 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 모델 리스트
models = {
    'Logistic Regression': LogisticRegression(),
    'Naive Bayes': GaussianNB()
}

# 모델 학습 및 평가 결과 저장을 위한 파일
results = []

# 모델 학습 및 평가
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results.append(f'{model_name} Accuracy: {accuracy:.4f}\n{report}\n')

# 결과를 txt 파일로 저장
with open('result/LR.NB_model_results_SMOTE.txt', 'w') as f:
    for result in results:
        f.write(result)
        f.write('\n')

print("LR.NB (SMOTE): Results have been saved to 'LR.NB_model_results_SMOTE.txt'.")

## 베이지안 옵티마이제이션

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import xgboost as xgb
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# 학습 및 테스트 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 베이지안 옵티마이제이션을 위한 함수 정의
def optimize_model(model, param_bounds):
    def model_evaluate(**params):
        for param in params:
            if param in ['max_depth', 'min_samples_split', 'n_estimators', 'num_leaves', 'n_neighbors', 'leaf_size']:
                params[param] = int(params[param])
        model.set_params(**params)
        return cross_val_score(model, X_train, y_train, scoring='accuracy', cv=5).mean()

    optimizer = BayesianOptimization(f=model_evaluate, pbounds=param_bounds, random_state=42)
    optimizer.maximize(init_points=5, n_iter=10)
    
    return optimizer.max['params']


# 모델 리스트와 파라미터 범위
models = {
    'Decision Tree': (DecisionTreeClassifier(), {
        'max_depth': (1, 1000),
        'min_samples_split': (2, 500) 
    }),
    'Random Forest': (RandomForestClassifier(), {
        'n_estimators': (10, 2000),
        'max_depth': (1, 1000),
        'min_samples_split': (2, 500)
    }),
    'XGBoost': (xgb.XGBClassifier(), {
        'n_estimators': (10, 2000),
        'max_depth': (1, 200), 
        'learning_rate': (0.001, 0.5)
    }),
    'LightGBM': (lgb.LGBMClassifier(), {
        'n_estimators': (10, 2000), 
        'num_leaves': (20, 1000),
        'learning_rate': (0.001, 0.5) 
    }),
    'KNN': (KNeighborsClassifier(), {
        'n_neighbors': (1, 500), 
        'leaf_size': (10, 1000) 
    }),
    'Logistic Regression': (LogisticRegression(), {
        'C': (0.001, 1000),
        
    }),
    'Naive Bayes': (GaussianNB(), {
        'var_smoothing': (1e-12, 1e-6)
    })
}

# 모델 학습 및 평가 결과 저장을 위한 파일
results = []

# 모델 학습 및 평가
for model_name, (model, param_bounds) in models.items():
    print(f"Tuning {model_name}...")
    best_params = optimize_model(model, param_bounds)
    # 정수형 파라미터를 다시 정수로 변환
    for param in best_params:
        if param in ['max_depth', 'min_samples_split', 'n_estimators', 'num_leaves', 'n_neighbors', 'leaf_size']:
            best_params[param] = int(best_params[param])
    model.set_params(**best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results.append(f'{model_name} Accuracy: {accuracy:.4f}\n{report}\nBest Params: {best_params}\n')

# 결과를 txt 파일로 저장
with open('result/model_results_SMOTE_optimized.txt', 'w') as f:
    for result in results:
        f.write(result)
        f.write('\n')

print("All models (SMOTE): Results have been saved to 'model_results_SMOTE_optimized.txt'.")



# SHAP

In [ ]:
import shap
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import numpy as np

def plot_combined_learning_curve(models, X, y, model_names, title):
    plt.figure(figsize=(10, 8))
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
    for color, model_name, model in zip(colors, model_names, models):
        train_sizes, train_scores, test_scores = learning_curve(
            model, X, y, scoring='accuracy'
        )
        test_scores_mean = np.mean(test_scores, axis=1)
        plt.plot(train_sizes, test_scores_mean, 'o-', color=color, label=f'{model_name} Test Accuracy')
    
    plt.title(title, fontsize=16)
    plt.xlabel('Training Steps', fontsize=14)
    plt.ylabel('Accuracy', fontsize=14)
    plt.legend(loc='best', fontsize=12)
    plt.grid(True)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.tight_layout()
    plt.show()

# Example data, you should replace these with your actual data
# X_resampled and y_resampled should be your data
# X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# First group of models
linear_and_knn_models = [
    LogisticRegression(),
    GaussianNB(),
    KNeighborsClassifier(),
    DecisionTreeClassifier()
]

linear_and_knn_model_names = [
    'Logistic Regression',
    'Naive Bayes',
    'KNN',
    'Decision Tree'
]

# Second group of models
tree_based_models = [
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    xgb.XGBClassifier(),
    lgb.LGBMClassifier()
]

tree_based_model_names = [
    'Decision Tree',
    'Random Forest',
    'XGBoost',
    'LightGBM'
]

# Plot learning curves for first group
#plot_combined_learning_curve(linear_and_knn_models, X_train, y_train, linear_and_knn_model_names, 'Learning Curves for Logistic Regression, Naive Bayes, KNN, Decision Tree')

# Plot learning curves for second group
#plot_combined_learning_curve(tree_based_models, X_train, y_train, tree_based_model_names, 'Learning Curves for Decision Tree, Random Forest, XGBoost, LightGBM')

# Function to plot SHAP values
def plot_shap_values(model, X, model_name):
    explainer = shap.Explainer(model, X)
    shap_values = explainer(X)
    shap.summary_plot(shap_values, X, plot_type="violin", show=False)
    plt.title(f'SHAP Summary Plot for {model_name}')
    plt.show()

# Fit models and plot SHAP values
for model, name in zip(linear_and_knn_models + tree_based_models, linear_and_knn_model_names + tree_based_model_names):
    model.fit(X_train, y_train)
    plot_shap_values(model, X_train, name)
